In [1]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

import time

#init display
from pynq.lib.arduino import Arduino_Displaycam
test = Arduino_Displaycam(base.ARDUINO)
test.init()
test.gpio()
test.pwm() 

0

In [5]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

from pynq.lib.arduino import Arduino_SPOT
device = Arduino_SPOT(base.ARDUINO,Arduino_SPOT)

In [6]:
device.clearWindow(1)

KeyboardInterrupt: 

In [4]:
from pynq.lib.arduino.spot_device import *
import asyncio

spot = SPOT(device)
spot.start()

# spot.device.setImageAddress()
# for i in range(0,1):
#     spot.device.snapPic(100,0)
    
# #pickle
# spot.radar.savePicture(spot.device.drawAddr, "spottyThotty")
# print('pickled')

# #unpickle
# spot.radar.openPicture(spot.device.drawAddr, "spottyThotty")
# print('unpickled')

# #clear buffer
# for slot in spot.device.drawAddr:
#     slot = bytes([0])
# print('cleared buffer')

# #draw
# spot.device.drawImage(100,0)
# print('redrawn')

# while(True):
#     print(spot.device.readFromGPS())
#     time.sleep(1)

import random
types = ['INTEREST', 'DANGER']
counter = 1
points = [Point('USER_{}'.format(i)) for i in range(0, 10)]
for i in points:
    i.tag = 'Obstacle_{}'.format(counter)
    i.createdBy = 'CB_{}'.format(counter)
    i.distance = 'DISTANCE_{}'.format(counter)
    i.type = types[random.randint(0, 1)]
    i.currentLocation = (random.randint(-30, 30), random.randint(-30, 30))
    counter += 1
    spot.radar.addPoint(i)
    

# #init test points to draw
# point0 = Point('Brandon')
# point0.tag = 'Hot'
# point0.createdBy = 'Beta Saurabh'
# point0.distance = 69
# point0.type = 'DANGER'
# point0.currentLocation = (20,0)

# point1 = Point('Saurabh')
# point1.tag = 'Weird'
# point1.createdBy = 'Alpha Brandon'
# point1.type = 'INTEREST'
# point1.distance = 420
# point1.currentLocation = (-15,15)


# print(len(spot.radar.points))
# spot.radar.addPoint(point0)
# spot.radar.addPoint(point1)
# print(len(spot.radar.points))

spot.radar.userLocation = (0,0)

#loop testing
# while(True):
#     spot.loop()
#     time.sleep(0.5)

'''
Types of events:
TOP
BOTTOM
CW
CCW
ERROR
'''

@asyncio.coroutine
def buttonPress(num):
    event = 'ERROR'
    twistCount = 0
    while True:
        yield from base.buttons[num].wait_for_value_async(1)
        if(num == 3): #01
            twistCount = 1
            while twistCount == 1: #01
                if base.buttons[2].read() and base.buttons[3].read():
                    twistCount = 2
            while twistCount == 2: #11
                if base.buttons[2].read() and base.buttons[3].read() == 0:
                    twistCount = 3
            while twistCount == 3: #10
                if base.buttons[2].read() == 0 and base.buttons[3].read() == 0:
                    twistCount = 4
                    event = 'CCW'
            yield from asyncio.sleep(0.1)
            base.leds[num].off
        elif(num == 2): #10
            twistCount = 1
            while twistCount == 1: #10
                if base.buttons[2].read() and base.buttons[3].read():
                    twistCount = 2
            while twistCount == 2: #11
                if base.buttons[2].read() == 0 and base.buttons[3].read():
                    twistCount = 3
            while twistCount == 3: #01
                if base.buttons[2].read() == 0 and base.buttons[3].read() == 0:
                    twistCount = 4
                    event = 'CW'
            while base.buttons[num].read() == 1:
                base.leds[num].toggle()
                yield from asyncio.sleep(0.1)
            base.leds[num].off
        elif(num == 1):
            event = 'TOP'
            while base.buttons[num].read():
                base.leds[num].toggle()
                yield from asyncio.sleep(0.1)
            base.leds[num].off()
        elif(num == 0):
            event = 'BOTTOM'
            while base.buttons[num].read():
                base.leds[num].toggle()
                yield from asyncio.sleep(0.1)
            base.leds[num].off()
            
        else:
            event = 'ERROR'
        spot.on_event(event)
        
tasks = [asyncio.ensure_future(buttonPress(i)) for i in range(4)]

KeyboardInterrupt: 

In [ ]:
# '''
# self.write_TAG(100, 310)
# self.write_DISTANCE(100, 360)
# self.write_CB(100, 410)
# self.write_TYPE()
# '''

# device.clearAll()
# time.sleep(0.05)
# point = spot.radar.points[2]
# device.drawInfoState()
# if(point.type == 'DANGER'):
#     device.write_CUSTOM(point.type, 200, 265, 0xf800) # Variable type
# else:
#     device.write_CUSTOM(point.type, 200, 265, 0x07e8) # Variable type
# device.write_CUSTOM(point.tag, 190, 315, 0xffff) # Variable tag
# device.write_CUSTOM(str(point.distance) + ' meters', 270, 365, 0xffff) # Variable distance
# device.write_CUSTOM(point.createdBy, 300, 415, 0xffff) # Variable c/b

In [ ]:
@asyncio.coroutine
def txRead():
    txc = 0
    while True:
        yield from asyncio.sleep(0.1)
        while(device.hasMessages()):
            val = device.readFromTX()
            txc += 1
            if(txc == 600):
                txc = 0
                break
#             print(val);

tasks.append(asyncio.ensure_future(txRead()))

In [ ]:
@asyncio.coroutine
def loop():
    while True:
        yield from asyncio.sleep(0.3)
        spot.loop()

tasks.append(asyncio.ensure_future(loop()))

In [ ]:
# @asyncio.coroutine
# def gpsSend():
#     while True:
#         yield from asyncio.sleep(3)
#         gpsLoc = device.readFromGPS()
#         if(gpsLoc[0] != None):
#             gpsString = "g," + str(round(gpsLoc[0], 6)) + "," + str(round(gpsLoc[1], 6))
#             print(gpsString)
#             device.writeToTX(4, gpsString)
# tasks.append(asyncio.ensure_future(gpsSend()))

In [ ]:
# import psutil

# @asyncio.coroutine
# def print_cpu_usage():
#     # Calculate the CPU utilisation by the amount of idle time
#     # each CPU has had in three second intervals
#     last_idle = [c.idle for c in psutil.cpu_times(percpu=True)]
#     while True:
#         yield from asyncio.sleep(3)
#         next_idle = [c.idle for c in psutil.cpu_times(percpu=True)]
#         usage = [(1-(c2-c1)/3) * 100 for c1,c2 in zip(last_idle, next_idle)]
#         print("CPU Usage: {0:3.2f}%, {1:3.2f}%".format(*usage))
#         last_idle = next_idle

# tasks.append(asyncio.ensure_future(print_cpu_usage()))

In [ ]:
if base.switches[0].read():
    print("Please set switch 0 low before running")
else:
#     spot.loop()
    base.switches[0].wait_for_value(1)